In [1]:
import pymysql
import requests
import pandas as pd
import numpy as np

### Se conecta con la BD MySQL AWS

In [2]:
con = pymysql.connect(
      host='database-proyecto-final.cpjrl88bhdul.us-east-1.rds.amazonaws.com',
      database='databaseuci',
      user='admin',
      password='datascience'
)

### Genera un df con el archivo `csv` "CHARTEVENTS"

In [3]:
df = pd.read_csv('C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\CHARTEVENTS.csv', encoding='UTF-8', delimiter=',', decimal='.')
df.head(3)

C:\Users\racev\AppData\Local\Temp\ipykernel_15240\1554866284.py:1: DtypeWarning: Columns (8,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\CHARTEVENTS.csv', encoding='UTF-8', delimiter=',', decimal='.')


,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,5279021,40124,126179,279554.0,223761,2130-02-04 04:00:00,2130-02-04 04:35:00,19085,95.9,95.9,?F,0.0,0.0,NaN,NaN
1,5279022,40124,126179,279554.0,224695,2130-02-04 04:25:00,2130-02-04 05:55:00,18999,2222221.7,2222221.7,cmH2O,0.0,0.0,NaN,NaN
2,5279023,40124,126179,279554.0,220210,2130-02-04 04:30:00,2130-02-04 04:43:00,21452,15.0,15.0,insp/min,0.0,0.0,NaN,NaN


In [4]:
df = df.replace(np.nan, 0)

### Inserta los registros del df "CHARTEVENTS" en la BD

In [5]:
sql_insert = ''' INSERT 
                   INTO  CHARTEVENTS
                         (row_id, subject_id, hadm_id, icustay_id, itemid, charttime, storetime, cgid, value, valuenum,
                          valueuom, warning, error, resultstatus, stopped)  
                 VALUES  (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) '''
num_reng = 0
cursor = con.cursor()
for i in range(0, len(df)):
    row = int(df.loc[i, 'row_id'])
    subj = int(df.loc[i, 'subject_id'])
    hadm = int(df.loc[i, 'hadm_id'])
    icus = int(df.loc[i, 'icustay_id'])
    itid = int(df.loc[i, 'itemid'])
    chtm = df.loc[i, 'charttime']
    sttm = df.loc[i, 'storetime']
    cgid = int(df.loc[i, 'cgid'])
    valu = df.loc[i, 'value']
    valn = int(df.loc[i, 'valuenum'])
    valm = df.loc[i, 'valueuom']
    warn = int(df.loc[i, 'warning'])
    erro = int(df.loc[i, 'error'])
    rest = df.loc[i, 'resultstatus']
    stop = df.loc[i, 'stopped']
    reg_s = (row, subj, hadm, icus, itid, chtm, sttm, cgid, valu, valn, valm, warn, erro, rest, stop)
    cursor.execute(sql_insert, reg_s)
    num_reng += 1
    if num_reng == 100:
        con.commit()
        num_reng = 0

KeyboardInterrupt: 

### Confirma la inserción de los registros a la BD

In [17]:
con.commit()

### Cierra el cursor y la conexion a la BD

In [18]:
cursor.close()
con.close()